# 4 - Live Morphing

This notebook shows how the harmonic interpolation is done and it's integration with the circular buffer.

In [1]:
# Enabling ipympl for interactive plots and styling the UI a bit
# get_ipython().run_line_magic('matplotlib', 'widget') # ipympl
get_ipython().run_cell_magic('html', '', '<style>.widget-readout { color: white; font-size: 1.2em; box-shadow: none !important ;} </style>') # style

import sys, os, datetime
# import pixiedust
import numpy as np
import pyaudio as pa
import matplotlib.pyplot as plt

sys.path.append('/sms-tools/software/models')
import utilFunctions as UF
import sineModel as SM
import stochasticModel as STM

from utils.structures import Sound
from dotmap import DotMap
from scipy.signal import resample, blackmanharris, triang, hanning
from scipy.fftpack import fft, ifft, fftshift
from scipy.io.wavfile import write
from ipywidgets import ( HTML, Layout, FloatSlider, Label, ToggleButton, Button, GridBox, interactive_output, interact )

# Constants
DEFAULT_SOUND_FILE_1 = '../data/sounds/violin-B3.wav'
DEFAULT_SOUND_FILE_2 = '../data/sounds/soprano-E4.wav'
ANALYSIS_OUTPUT_FOLDER = '../data/analysis_output'
MORPHINGS_OUTPUT_FOLDER = '../data/morphing_output'

NS = 512 # size of fft used in synthesis
H = int(NS/4) # hop size (has to be 1/4 of NS) - 128
NUMBER_OF_FFT_SYNTH_FRAMES = 4 # number of FFT synth frames
FRAME_TO_PLOT = 0 # 100 # frame to plot
NUMBER_OF_FRAMES_TO_PLOT  = 8 # number of frames to plots
GENERATE_PLOTS = False # generate the plots

DARK_MODE = False

if (DARK_MODE):
    params = {
        "text.color" : "w",
        "ytick.color" : "w",
        "xtick.color" : "w",
        "axes.labelcolor" : "w",
        "axes.edgecolor" : "w",
#         "axes.facecolor" : 'e5e5e5'
    }
    plt.rcParams.update(params)

In [2]:
# Load the sounds by default
sound_1 = Sound(DEFAULT_SOUND_FILE_1)
sound_2 = Sound(DEFAULT_SOUND_FILE_2)

# Loading the .had files
sound_1.load_had_file()
sound_2.load_had_file()

# Empty sound where the morph will be generated 
sound_morph = Sound(DEFAULT_SOUND_FILE_2)

# GUI elements that does not contain values about the sound
gui = DotMap()

In [3]:
def interpolateFrames(frame_1, frame_2, harmonics, interp_factor):
    
    interpolated_frame = (1-interp_factor) * frame_1[harmonics] + interp_factor * frame_2[harmonics]
    
    return interpolated_frame

In [4]:
def plotCircularBuffer(i, mCircularBufferLeft, mCircularBufferWriteHead, mCircularBufferPlayHead, headWritePointer, tailWritePointer, tailCleanPointer, headPlayPointer, tailPlayPointer):

    fig = plt.figure(num='mCircularBufferLeft', figsize=(14, 6))
    plt.title('mCircularBufferLeft i=' + str(i))
    plt.plot(mCircularBufferLeft)

    # If the tail celan pointer is ahead the tail write pointer
    if ( (tailCleanPointer-tailWritePointer) < 0 ):
        plt.axvspan(tailWritePointer, mCircularBufferLength, facecolor='r', alpha=0.5)
        plt.axvspan(0, tailCleanPointer, facecolor='r', alpha=0.5)
    else:
        plt.axvspan(tailWritePointer, tailCleanPointer, facecolor='r', alpha=0.5)

    # If the tail pointer is ahead the head pointer
    if ( (tailWritePointer-headWritePointer) < 0 ):
        plt.axvspan(headWritePointer, mCircularBufferLength, facecolor='orange', alpha=0.5)
        plt.axvspan(0, tailWritePointer, facecolor='orange', alpha=0.5)
    else:
        plt.axvspan(headWritePointer, tailWritePointer, facecolor='orange', alpha=0.5)

    # If the tail pointer is ahead the head pointer
    if ( (tailPlayPointer-headPlayPointer) < 0 ):
        plt.axvspan(headPlayPointer, mCircularBufferLength, facecolor='g', alpha=0.5)
        plt.axvspan(0, tailPlayPointer, facecolor='g', alpha=0.5)
    else:
        plt.axvspan(headPlayPointer, tailPlayPointer, facecolor='g', alpha=0.5)
    
    axes = plt.gca()
    axes.set_ylim([-0.15,0.20])
    plt.show()
    fig.savefig('images/circular_buffer_' + str(i) + '.png', transparent=True, dpi=fig.dpi*4)

In [5]:
def plotHarmonics(sound_name, sound_freqs, all_harmonics, max_harmonics, freqs_interp_factor = None, mags_interp_factor = None, stocs_interp_factor = None):
    
    aux_freqs_1 = sound_freqs
    aux_freqs_1[ aux_freqs_1==0 ] = np.nan
    fig = plt.figure(num='Freqs Sound ' + sound_name, figsize=(14, 9))
    plt.title('Freqs Sound ' + sound_name)
    if (DARK_MODE): plt.plot(aux_freqs_1, 'w')
    else: plt.plot(aux_freqs_1,'black')
    i = 0
    aux_sel_freq_1 = []
    while i < len(all_harmonics):
        aux_sel_freq_1.append([])
        i_harmonic = 0
        while i_harmonic < max_harmonics:
            if i_harmonic in all_harmonics[i]:
                aux_sel_freq_1[i].append(sound_freqs[i][i_harmonic])
            else:
                aux_sel_freq_1[i].append(np.nan)
            i_harmonic += 1
        i += 1
    plt.plot(aux_sel_freq_1, 'r')
    if (DARK_MODE): plt.gca().set_facecolor('#373e4b')
    plt.show()
    
    if (freqs_interp_factor != None and
        mags_interp_factor != None and 
        stocs_interp_factor != None ):
        
        file_name = 'harmonics_' + sound_name + \
                    '_' + str(freqs_interp_factor) + \
                    '_' + str(mags_interp_factor) + \
                    '_' + str(stocs_interp_factor)
    else:
        file_name = 'harmonics_' + sound_name
        
    fig.savefig('images/' + file_name + '.png', transparent=True, dpi=fig.dpi*4)

In [6]:
def runMorph(freqs_interp_factor, mags_interp_factor, stocs_interp_factor):
# def runMorph(gui, sound_1, sound_2, sound_morph):
    try:
        # Get the original values from the sound 1
        o_freqs_1 = sound_1.analysis.output.values.hfreq
        o_mags_1 = sound_1.analysis.output.values.hmag
        o_stocs_1 = sound_1.analysis.output.values.stocEnv

        # Get the original values from the sound 2
        o_freqs_2 = sound_2.analysis.output.values.hfreq
        o_mags_2 = sound_2.analysis.output.values.hmag
        o_stocs_2 = sound_2.analysis.output.values.stocEnv

        # Get the maximum overall shape (length, number of harmonics)
        max_len, max_harmonics = np.maximum(o_freqs_1.shape, o_freqs_2.shape)
        _, max_stochastic = np.maximum(o_stocs_1.shape, o_stocs_2.shape)

        # Initialize the frequencies arrays with zeros
        freqs_1 = np.zeros((max_len, max_harmonics))
        freqs_2 = np.zeros((max_len, max_harmonics))
        freqs_morph = np.zeros((max_len, max_harmonics))

        # Initialize the magnitudes arrays with zeros
        mags_1 = np.full((max_len, max_harmonics), -100)
        mags_2 = np.full((max_len, max_harmonics), -100)
        mags_morph = np.full((max_len, max_harmonics), -100)

        # Initialize the stochastic values arrays with zeros
        stocs_1 = np.zeros((max_len, max_stochastic))
        stocs_2 = np.zeros((max_len, max_stochastic))
        stocs_morph = np.zeros((max_len, max_stochastic))

        # Fill the frequencies arrays values
        freqs_1[:o_freqs_1.shape[0],:o_freqs_1.shape[1]] = o_freqs_1
        freqs_2[:o_freqs_2.shape[0],:o_freqs_2.shape[1]] = o_freqs_2

        # Fill the magnitudes arrays values
        mags_1[:o_mags_1.shape[0],:o_mags_1.shape[1]] = o_mags_1
        mags_2[:o_mags_2.shape[0],:o_mags_2.shape[1]] = o_mags_2

        # Fill the stochastic arrays values
        stocs_1[:o_stocs_1.shape[0],:o_stocs_1.shape[1]] = o_stocs_1
        stocs_2[:o_stocs_2.shape[0],:o_stocs_2.shape[1]] = o_stocs_2
        
        lastytfreq = freqs_1[0,:]
        phase_morph = 2*np.pi*np.random.rand(freqs_1[0,:].size)
        phases_morph = np.array([])
        all_harmonics = []

        # Define the window
        ow = triang(2*H)
        bh = blackmanharris(NS)
        hN = NS//2
        sw = np.zeros(NS)
        sw[hN-H:hN+H] = ow
        bh = bh / sum(bh)
        sw[hN-H:hN+H] = sw[hN-H:hN+H]/bh[hN-H:hN+H]

        if (GENERATE_PLOTS):
            plt.figure(num='window_applied', figsize=(14, 6))
            plt.title('Window Applied')
            plt.plot(sw)
            plt.show()
        
        # Calculate the circular buffer length
        mCircularBufferLength = NS * NUMBER_OF_FFT_SYNTH_FRAMES
        
        # Circular Buffer
        mCircularBufferLeft = np.zeros(mCircularBufferLength, dtype = np.float32)
        mCircularBufferRight = np.zeros(mCircularBufferLength, dtype = np.float32)

        frame_to_play = np.array([])
        full_sound = np.array([])
        
        # Initialize the write and play heads to 0
        mCircularBufferWriteHead = 0;
        mCircularBufferPlayHead = 0;

        pya = pa.PyAudio()
        stream = pya.open(format=pa.paFloat32,
                      channels=1,
                      rate=sound_morph.fs,
                      output=True)

    except Exception as errorMessage:

        print("Initialization error")
        
        # Displaying the error
        print(str(errorMessage))
            
    # For each frame in the audio file
    for i in range(max_len):

        # Identify harmonics that are present in both frames
        harmonics = np.intersect1d(
            np.array(np.nonzero(freqs_1[i]), dtype=np.int)[0],
            np.array(np.nonzero(freqs_2[i]), dtype=np.int)[0])
        
        # Save all the chosen harmonics in a list
        all_harmonics.append(harmonics)

        # Interpolating the frequencies of the mathcing harmonics
        freqs_morph[i][harmonics] = interpolateFrames(
            freqs_1[i], freqs_2[i], harmonics, freqs_interp_factor)

        # Interpolating the magnitudes of the mathcing harmonics
        mags_morph[i][harmonics] = interpolateFrames(
            mags_1[i], mags_2[i], harmonics, mags_interp_factor)

        stoc_components = np.arange(0,max_stochastic)

        # Interpolating the stochastic values of the mathcing harmonics
        stocs_morph[i][stoc_components] = interpolateFrames(
            stocs_1[i], stocs_2[i], stoc_components, stocs_interp_factor)
        
        phase_morph += (np.pi*(lastytfreq+freqs_morph[i,:])/sound_morph.fs)*H
        
        # Keep phase inside 2*pi
        phase_morph = phase_morph % (2*np.pi)
        phases_morph = np.append(phases_morph,phase_morph)
        
        # Generate sines in the spectrum
        Y = UF.genSpecSines(freqs_morph[i], mags_morph[i], phase_morph, NS, sound_morph.fs)
        
        # Save the current frequencies to be available fot the next iteration
        lastytfreq = freqs_morph[i,:]
        
        # Compute inverse FFT
        yw = np.real(fftshift(ifft(Y)))
        
        # Stochastic component (WIP) 
#         yw_stoc_s = stochasticModelSynth(stocs_morph[i], H, H*2)
#         yw_stoc = np.real(ifft(yw_stoc_s))

        # Keep the pointers inside the circular buffer 
        headWritePointer = mCircularBufferWriteHead % mCircularBufferLength
        tailWritePointer = (mCircularBufferWriteHead+NS) % mCircularBufferLength
        tailCleanPointer = (mCircularBufferWriteHead+NS+H) % mCircularBufferLength

        # If the tail celan pointer is ahead the tail write pointer
        if ( (tailCleanPointer-tailWritePointer) < 0 ):
            buffer_samples_to_clean = np.r_[tailWritePointer:mCircularBufferLength,0:tailCleanPointer]
        else:
            buffer_samples_to_clean = np.r_[tailWritePointer:tailCleanPointer]

        # Clean the part of the buffer that we must overrid
        mCircularBufferLeft[buffer_samples_to_clean] = 0

        # If the tail pointer is ahead the head pointer
        if ( (tailWritePointer-headWritePointer) < 0 ):
            selected_buffer_samples = np.r_[headWritePointer:mCircularBufferLength,0:tailWritePointer]      
        else:
            selected_buffer_samples = np.r_[headWritePointer:tailWritePointer]
        
        # Apply the window
        mCircularBufferLeft[selected_buffer_samples] += sw*yw
        
        # Enter this section when a chunk of the audio buffer is ready to be played
        if (i % (NS/H) == 0): # i % 4
        
            # Keep the pointers inside the circular buffer 
            headPlayPointer = mCircularBufferPlayHead % mCircularBufferLength
            tailPlayPointer = (mCircularBufferPlayHead+NS) % mCircularBufferLength

            # If the tail pointer is ahead the head pointer
            if ( (tailPlayPointer-headPlayPointer) < 0 ):
                selected_buffer_samples = np.r_[headPlayPointer:mCircularBufferLength,0:tailPlayPointer]
            else:
                selected_buffer_samples = np.r_[headPlayPointer:tailPlayPointer]

            frame_to_play = mCircularBufferLeft[selected_buffer_samples]

            frame_to_play_scaled = frame_to_play.astype(np.float32).tostring()
            
            # Live Stream
            stream.write(frame_to_play_scaled)

            # Write sound file
            full_sound = np.concatenate((full_sound,frame_to_play))

        # Keep the pointers inside the circular buffer length
        if (mCircularBufferWriteHead >= mCircularBufferLength):
            mCircularBufferWriteHead = 0;

        if (mCircularBufferPlayHead >= mCircularBufferLength):
            mCircularBufferPlayHead = 0;

        if (GENERATE_PLOTS and i >= FRAME_TO_PLOT and
            i <= FRAME_TO_PLOT + NUMBER_OF_FRAMES_TO_PLOT):

            plotCircularBuffer(
                i, mCircularBufferLeft,
                mCircularBufferWriteHead, mCircularBufferPlayHead,
                headWritePointer, tailWritePointer, tailCleanPointer,
                headPlayPointer, tailPlayPointer)

        mCircularBufferWriteHead += H
        mCircularBufferPlayHead = mCircularBufferWriteHead - NS
        
        if (mCircularBufferPlayHead < 0):
            mCircularBufferPlayHead = mCircularBufferPlayHead % mCircularBufferLength;

    # Close the sound stream
    stream.stop_stream()
    stream.close()

    current_timestamp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"))

    # Write a file with the live generated sound
    write(MORPHINGS_OUTPUT_FOLDER + '/morph' + \
          '_' + str(freqs_interp_factor) + \
          '_' + str(mags_interp_factor) + \
          '_' + str(stocs_interp_factor) + \
          '_' + current_timestamp + '.wav', sound_morph.fs, full_sound)

    # Generate the whole sound with the freqs and mags arrays with the sms-tools function
    yh = SM.sineModelSynth(freqs_morph, mags_morph, np.array([]), NS, H, sound_morph.fs)
    yst = STM.stochasticModelSynth(stocs_morph, H, H*2)
    sms_sound = yh[:min(yh.size, yst.size)] + yst[:min(yh.size, yst.size)]

    # Write a file with the sms-tools generated sound
    write(MORPHINGS_OUTPUT_FOLDER + '/morph_sms_' + current_timestamp + '.wav', sound_morph.fs, sms_sound)

    if (GENERATE_PLOTS):
        # Plot Sound 1 chosen harmonics
        plotHarmonics('1', freqs_1, all_harmonics, max_harmonics)

        # Plot Sound 2 chosen harmonics
        plotHarmonics('2', freqs_2, all_harmonics, max_harmonics)
        
        # Plot Sound Morph harmonics
        plotHarmonics('Morph', freqs_morph, all_harmonics, max_harmonics,
                      freqs_interp_factor, mags_interp_factor, stocs_interp_factor)

#     plt.figure(num='phases_morph', figsize=(14, 6))
#     plt.title('phases_morph')
#     plt.plot(phases_morph[:1000])
#     plt.show()

#     plt.figure(num='Full Sound', figsize=(14, 6))
#     plt.title('Full Sound')
#     plt.plot(full_sound)
#     plt.show()

#     plt.figure(num='sms_sound', figsize=(14, 6))
#     plt.title('sms_sound')
#     plt.plot(sms_sound)
#     plt.show()

In [7]:
# Morph Controls UI

# Morph Controls - Header
gui.morph_controls_header = HTML(
     value="<center style='color:#b1bed6;font-size:1.5em;margin-top:1em;overflow:hidden'>Morph Controls</center>",
     layout=Layout(width='auto', grid_area='morph_controls_header')
)

# Morph Controls - Sub-Header
gui.morph_controls_sub_header = HTML(
     value="<center style='color:#b1bed6;font-size:1.2em;margin-top:1em;overflow:hidden'>Interpolation Factors - 0 to 1 (time, value pairs)</center>",
     layout=Layout(width='auto', grid_area='morph_controls_sub_header')
)

# Morph Controls - Harmonic Frequencies
sound_morph.harmonic_frequencies = FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    continuous_update = True,
    layout=Layout(width='auto', grid_area='morph_harmonic_frequencies')
)
gui.morph_harmonic_frequencies_label = Label(
    value='Harmonic Frequencies',
    layout=Layout(width='auto', grid_area='morph_harmonic_frequencies_label')
)

# Morph Controls - Harmonic Magnitudes
sound_morph.harmonic_magnitudes = FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    continuous_update = True,
    layout=Layout(width='auto', grid_area='morph_harmonic_magnitudes')
)
gui.morph_harmonic_magnitudes_label = Label(
    value='Harmonic Magnitudes',
    layout=Layout(width='auto', grid_area='morph_harmonic_magnitudes_label')
)

# Morph Controls - Stochastic Component
sound_morph.stochastic_component = FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    continuous_update = True,
    layout=Layout(width='auto', grid_area='morph_stochastic_component')
)
gui.morph_stochastic_component_label = Label(
    value='Stochastic Component',
    layout=Layout(width='auto', grid_area='morph_stochastic_component_label')
)

# Morph Controls - Run/Stop Morph Button
gui.run_morph_button = ToggleButton(
    value=False,
    description='Run',
    button_style='info',
    layout=Layout(width='auto', grid_area='run_morph_button'),
)

# Morph Message
gui.morph_message = Button(
    description="",
    button_style='danger',
    layout=Layout(width='auto', visibility='hidden', grid_area='morph_message')
)

# Morph Controls - Panel
morph_controls_panel = GridBox(
    children=[
        gui.morph_controls_header, gui.morph_controls_sub_header,
        gui.morph_harmonic_frequencies_label, sound_morph.harmonic_frequencies,
        gui.morph_harmonic_magnitudes_label, sound_morph.harmonic_magnitudes,
        gui.morph_stochastic_component_label, sound_morph.stochastic_component,
        gui.morph_message
    ],
    layout=Layout(
        width='auto',
        grid_area='morph_controls_panel',
        grid_template_rows='auto auto auto auto',
        grid_template_columns='25% 25% 25% 25%',
        grid_template_areas='''
        "morph_controls_header morph_controls_header morph_controls_header morph_controls_header"
        "morph_controls_sub_header morph_controls_sub_header morph_controls_sub_header morph_controls_sub_header"
        "morph_harmonic_frequencies_label morph_harmonic_frequencies morph_harmonic_frequencies ."
        "morph_harmonic_magnitudes_label morph_harmonic_magnitudes morph_harmonic_magnitudes ."
        "morph_stochastic_component_label morph_stochastic_component morph_stochastic_component ."
        ". morph_message morph_message ."
        '''
    )
)

# Mounting the GUI
grid = GridBox(
    children=[
        morph_controls_panel
    ],
    layout=Layout(
        width='100%',
        grid_template_rows='auto auto auto auto',
        grid_template_columns='25% 25% 25% 25%',
        grid_template_areas='''
        "morph_controls_panel morph_controls_panel morph_controls_panel morph_controls_panel"
        '''
    )
)

display(grid)

if (GENERATE_PLOTS):
    # Run this code to see the plots
    runMorph(
        sound_morph.harmonic_frequencies.value,
        sound_morph.harmonic_magnitudes.value,
        sound_morph.stochastic_component.value
    )

else:
    # Run this code for interactivity
    interactive_output(runMorph, {
        "freqs_interp_factor": sound_morph.harmonic_frequencies,
        "mags_interp_factor": sound_morph.harmonic_magnitudes,
        "stocs_interp_factor": sound_morph.stochastic_component
    });

GridBox(children=(GridBox(children=(HTML(value="<center style='color:#b1bed6;font-size:1.5em;margin-top:1em;ov…